<a id='top'></a>

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# %matplotlib nbagg 
%matplotlib notebook
# %matplotlib inline



## Third party 
import numpy as np
import os, time, zarr, sys
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl

import unslice.IO as io
from unslice.utils import *
from unslice.registration.featmatch import *
from unslice.registration.transform import *
from unslice.registration.rigid import *
from unslice.registration.gpu_transform import *
from unslice.registration.utils import *
from unslice.segmentation import *
from unslice.tracing.pyoof import OOF, apply_oof_v2
from unslice.tracing.skel import *
from unslice.flatten import *
from unslice.lightsheetcorrect import *




In [2]:
# Parameters that are constant throughout notebook
working_dir = '/mnt/share3/webster/mEhmAD_1-3_real'

def bdir(fname):
    return os.path.join(working_dir, fname)

# prefix to add to the beginning of each filename 
name_prefix = '1-abeta_cropped' 
name_prefix2 = '3-abeta_cropped'

# Table of contents

### Pre-processing
[1. Convert to zarr](#convert)<br>
[2. Flatten warp](#flattenwarp)<br>
[3. Ptau warp](#anchorwarp)<br>

# Convert to zarr
<a id='convert'></a>

[Return to top](#top)

### slab 1

In [4]:
# FIgure out where to crop

xrange = [5759,8410]
yrange = [5981,7328]
zrange = [4213-1500,4213+500] 
warped_zarr_path = '/mnt/share3/webster/mEhmAD_1-3_real/1-ptau_flattened.zarr'
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/1-lec_grid_flatten_upsampled.npy'


# We can find true "bounding box" by warping a bunch of points 
num_pts = 8
X,Y,Z = np.meshgrid(np.linspace(xrange[0],xrange[1],num_pts),
           np.linspace(yrange[0],yrange[1],num_pts),
           np.linspace(zrange[0],zrange[1],num_pts))

grid_pts = np.vstack((X.ravel(),Y.ravel(),Z.ravel())).T
bbox = grid_transform_pts(grid_path, grid_pts, warped_zarr_path, save_path=None, save_json=False, inverse_transform=False)

In [5]:
# Below are the coordinates in the original non-flattened field of view 

print(bbox[:,0].min(),bbox[:,0].max())
print(bbox[:,1].min(),bbox[:,1].max())
print(bbox[:,2].min(),bbox[:,2].max())

5762.127489409635 8418.448379403882
5920.223308435901 7254.6456738491015
1990.6485062161546 4354.164263091231


In [9]:
# 1 

slab_path = '/mnt/cephfs/general/juhyuk/15xdata/20220513_09_59_28_mEhmAD_1_abeta.lec.ptau/Ex_488_Em_1_destriped_stitched_tiffs'
slab_zarr_path = bdir(name_prefix+'.zarr')
pc2_img_size = (24992,9756,5935) #(18100,8100,5336) 


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (0.306,0.433,0.306)
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
coord0s = [2500,1000,497] # this is how much to add to coordinates 
# This is the range in the flattened frame that we cropped
xrange = [5762,8419]; xrange = [xrange[k]+coord0s[0] for k in range(2)]
yrange = [5920,7255]; yrange = [yrange[k]+coord0s[1] for k in range(2)]
zrange = [1990,4355]; zrange = [zrange[k]+coord0s[2] for k in range(2)]

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:8262-10919, y:6920-8255, z:2487-2527


100%|██████████| 40/40 [01:01<00:00,  1.54s/it]

Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...



100%|██████████| 98/98 [00:01<00:00, 55.53it/s]

Processing chunk x:8262-10919, y:6920-8255, z:2527-2567


...

Processing chunk x:8262-10919, y:6920-8255, z:4847-4852


100%|██████████| 5/5 [00:08<00:00,  1.80s/it]

Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...



100%|██████████| 98/98 [00:10<00:00,  9.00it/s]

4023.172872066498 seconds elapsed


## slab 3

In [6]:
# FIgure out where to crop

xrange = [4845,7742]
yrange = [5816,7372]
zrange = [501,2415]
warped_zarr_path = '/mnt/share3/webster/mEhmAD_1-3_real/3-ptau_flattened.zarr'
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/3-lec_grid_flatten_upsampled.npy'


# We can find true "bounding box" by warping a bunch of points 
num_pts = 8
X,Y,Z = np.meshgrid(np.linspace(xrange[0],xrange[1],num_pts),
           np.linspace(yrange[0],yrange[1],num_pts),
           np.linspace(zrange[0],zrange[1],num_pts))

grid_pts = np.vstack((X.ravel(),Y.ravel(),Z.ravel())).T
bbox = grid_transform_pts(grid_path, grid_pts, warped_zarr_path, save_path=None, save_json=False, inverse_transform=False)

In [7]:
# Below are the coordinates in the original non-flattened field of view 

print(bbox[:,0].min(),bbox[:,0].max())
print(bbox[:,1].min(),bbox[:,1].max())
print(bbox[:,2].min(),bbox[:,2].max())

4830.002511645537 7740.160110111759
5725.033439068361 7303.192936711223
-32.55155261351463 2158.8236936957096


In [8]:
# 3

slab_path = '/mnt/cephfs/general/juhyuk/15xdata/20220516_22_11_41_mEhmAD_3_abeta.lec.ptau/Ex_488_Em_1_destriped_stitched_tiffs'
slab_zarr_path = bdir(name_prefix2+'.zarr')
pc2_img_size = (21268,9755,5918) #(22177,7817,5933) this is the mehmad2 size


## Optional parameters 
load_num_slices = 40 # should be smaller than z chunk size, default None 
resample_num_slices = 1 # number of slices to be processed in one go for resampling, default 1
resample_factor = (1,1) # original is (0.306,0.433,0.306)
chunks = (200,200,200) 
num_workers = 20 
file_names = 'img_[0-9]{4}.tiff' # default 'img_[0-9]{4}.tiff'

## crop
coord0s = [2740,1096,862] # this is how much to add to coordinates 
# This is the range in the unflattened image, but cropped from original (calculated above)
xrange = [4830,7741]; xrange = [xrange[k]+coord0s[0] for k in range(2)]
yrange = [5725,7304]; yrange = [yrange[k]+coord0s[1] for k in range(2)]
zrange = [0,2159]; zrange = [zrange[k]+coord0s[2] for k in range(2)]

## rotate
lateral_rotate_angle = None # default None

## flip
flip = (0,0,0) # default (0,0,0)


#############
start = time.time()
utils.convert_to_zarr_v2(slab_path, slab_zarr_path, pc2_img_size, load_num_slices=load_num_slices,
                        resample_num_slices=resample_num_slices, file_names=file_names, 
                        chunks=chunks, num_workers=num_workers, lateral_rotate_angle=lateral_rotate_angle,
                        flip=flip, crop_xcoords=xrange, crop_ycoords=yrange, crop_zcoords=zrange,
                        resample_factor=resample_factor)
print(time.time()-start, 'seconds elapsed')

Processing chunk x:7570-10481, y:6821-8400, z:862-902


100%|██████████| 40/40 [00:45<00:00,  1.15s/it]

Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...



100%|██████████| 120/120 [00:02<00:00, 54.55it/s]


Processing chunk x:7570-10481, y:6821-8400, z:902-942


...

Processing chunk x:7570-10481, y:6821-8400, z:2982-3021


100%|██████████| 39/39 [00:50<00:00,  1.29s/it]

Data I/O, resampling, rotation complete. Commencing flip and zarr assignment...



100%|██████████| 120/120 [00:12<00:00,  9.95it/s]

3277.8234803676605 seconds elapsed


## Flatten warp the slabs

[Return to top](#top)

<a id='flattenwarp'></a>

## Crop flattening grids

### (1)

In [10]:
# (1)
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/1-lec_grid_flatten_upsampled.npy'
original_shape = (18100,8100,5336) # for which grid was made
new_grid_spacing = 3*(64,) # doesn't matter if range_mode= 'moving'
range_mode='moving'

# We will compute  the grid spacing from the zarr specifically 
save_grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/1-lec_grid_flatten_upsampled_cropped_abeta.npy'

# these are the coordinates in original image of FOV 
xrange = [5762,8419]
yrange = [5920,7255]
zrange = [1990,4355]


################
grid_new,new_shape = crop_grid(grid_path, original_shape, new_grid_spacing, range_mode=range_mode, 
                         xrange=xrange, yrange=yrange, zrange=zrange, save_grid_path=save_grid_path)

New fixed image shape: (2706, 1491, 2798)
New fixed range: x 5732 8438
New fixed range: y 5896 7387
New fixed range: z 2342 5140


### Coordinates
New fixed range: x 5732 8438<br>
New fixed range: y 5896 7387<br>
New fixed range: z 2342 5140<br>

### coordinates of the PTAU cropped image
xrange = [5759,8410] <br>
yrange = [5981,7328] <br>
zrange = [4213-1500,4213+500] 

In [11]:
# 1
fixed_zarr_path = new_shape
moving_zarr_path = bdir(name_prefix+'.zarr')
warped_zarr_path = bdir(name_prefix+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(64,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/1-lec_grid_flatten_upsampled_cropped_abeta.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = new_shape[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(2706, 1491, 2798)
Loading grid values...
Warping image...
Moving image size: 16.77775935 GB


100%|██████████| 1568/1568 [02:55<00:00,  8.91it/s]


Time elapsed: 5.089859 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:17<00:00, 11.20it/s]


Loading z 200 - 400


...

100%|██████████| 200/200 [00:16<00:00, 11.82it/s]


Loading z 2400 - 2600


100%|██████████| 200/200 [00:18<00:00, 10.89it/s]


Loading z 2600 - 2798


100%|██████████| 198/198 [00:17<00:00, 11.35it/s]


### (3)

In [12]:
# (3)
grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/3-lec_grid_flatten_upsampled.npy'
original_shape = (16172,8004,5052)
new_grid_spacing = 3*(64,) # doesn't matter if range_mode= 'moving'
range_mode='moving'

# We will compute  the grid spacing from the zarr specifically 
save_grid_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/3-lec_grid_flatten_upsampled_cropped_abeta.npy'

# these are the coordinates in original image of FOV 
xrange = [4830,7741]
yrange = [5725,7304]
zrange = [0,2159]




################
grid_new,new_shape = crop_grid(grid_path, original_shape, new_grid_spacing, range_mode=range_mode, 
                         xrange=xrange, yrange=yrange, zrange=zrange, save_grid_path=save_grid_path)

New fixed image shape: (3029, 1743, 2756)
New fixed range: x 4767 7796
New fixed range: y 5744 7487
New fixed range: z 65 2821


In [13]:
# 1
fixed_zarr_path = new_shape
moving_zarr_path = bdir(name_prefix2+'.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(64,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/3-lec_grid_flatten_upsampled_cropped_abeta.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = new_shape[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(3029, 1743, 2756)
Loading grid values...
Warping image...
Moving image size: 19.847553142 GB


100%|██████████| 2016/2016 [03:29<00:00,  9.63it/s]


Time elapsed: 6.319159 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:23<00:00,  8.69it/s]


Loading z 200 - 400


...

100%|██████████| 200/200 [00:21<00:00,  9.29it/s]


Loading z 2400 - 2600


100%|██████████| 200/200 [00:21<00:00,  9.17it/s]


Loading z 2600 - 2756


100%|██████████| 156/156 [00:16<00:00,  9.24it/s]


## TPS warp using ptau grid

[Return to top](#top)

<a id='anchorwarp'></a>

## Crop ptau grid

### In flattened frame, these are our ABETA coordinates
New fixed range: x 4767 7796<br>
New fixed range: y 5744 7487<br>
New fixed range: z 65 2821<br>

### In flattend rame, these are the PTAU coordinates for which the grid is for
xrange = [4845,7742]<br>
yrange = [5816,7372]<br>
zrange = [501,2415]<br>

In [15]:
# We need to crop the flattened to be the same size as the ptau flattened zarr to be warped

# bounding box of moving in flat frame
xrange = [4845-4767,7742-4767]
yrange = [5816-5744,7372-5744]
zrange = [501-65,2415-65]
num_workers = 24
source_zarr_path = bdir(name_prefix2+'_flattened.zarr')
sink_zarr_path = bdir(name_prefix2+'_flattened_recropped.zarr')



###############
crop_zarr(source_zarr_path, sink_zarr_path, xrange=xrange, yrange=yrange, zrange=zrange, 
          load_num_slices=None, num_workers=num_workers)

Processing chunk x:78-2975, y:72-1628, z:436-636


100%|██████████| 120/120 [00:07<00:00, 15.79it/s]


Processing chunk x:78-2975, y:72-1628, z:636-836


100%|██████████| 120/120 [00:08<00:00, 14.58it/s]


Processing chunk x:78-2975, y:72-1628, z:836-1036


...

100%|██████████| 120/120 [00:08<00:00, 13.36it/s]


Processing chunk x:78-2975, y:72-1628, z:2036-2236


100%|██████████| 120/120 [00:08<00:00, 14.63it/s]


Processing chunk x:78-2975, y:72-1628, z:2236-2350


100%|██████████| 120/120 [00:05<00:00, 20.89it/s]


In [3]:
# 3
fixed_zarr_path = (2651,1347,2000)
moving_zarr_path = bdir(name_prefix2+'_flattened_recropped.zarr')
warped_zarr_path = bdir(name_prefix2+'_flattened_recropped_ptauwarp_new_r2.zarr')


# Parameters for TPS zarr warp
grid_spacing = 3*(16,)
chunks=3*(200,)
nb_workers = 8


# grid I/O 
save_grid_values_path = None
use_grid_values_path = '/mnt/share3/webster/mEhmAD_1-3_real/warping_grids/grid_ptauwarp_cropped_new_r2.npy'




##########################

moving_pts_paths = None # doesn't actually matter for this 
fixed_pts_paths =  None

# anchor parameters (using the surface on the other side and manually identified anchors on the cut surface)
static_pts_paths = None
# affine parameters 
R_path = None
b_path = None
zadd = fixed_zarr_path[2]-zarr.open(moving_zarr_path).shape[2]

TPS_warp(moving_zarr_path, fixed_zarr_path, warped_zarr_path, moving_pts_paths, fixed_pts_paths,
         static_pts_paths=static_pts_paths, R_path=R_path, b_path=b_path,
          grid_spacing=grid_spacing, smooth=2, chunks=chunks,zadd=zadd,
          nb_workers=nb_workers, padding=2, save_grid_values_path=save_grid_values_path, 
          show_residuals=True, use_grid_values_path=use_grid_values_path)

# Convert zarr to tiff
tiff_path = warped_zarr_path[:-5]+'_tiffs'
convert_zarr_to_tiff(warped_zarr_path, tiff_path, num_workers=24)

(2651, 1347, 2000)
Loading grid values...
Warping image...
Moving image size: 17.255598096 GB


100%|██████████| 980/980 [02:23<00:00,  6.82it/s]


Time elapsed: 4.838330 minutes
Loading z 0 - 200


100%|██████████| 200/200 [00:16<00:00, 12.12it/s]


Loading z 200 - 400


...

100%|██████████| 200/200 [00:16<00:00, 12.43it/s]


Loading z 1600 - 1800


100%|██████████| 200/200 [00:15<00:00, 12.55it/s]


Loading z 1800 - 2000


100%|██████████| 200/200 [00:15<00:00, 13.03it/s]
